In [0]:
use catalog uc_prod

In [0]:
select distinct productgroup_name,product_name,record_updated_date from silver.daily_pricing_silver
where lower(product_name) = 'onion'

In [0]:
select *  from gold.reporting_dim_product_gold
where lower(PRODUCT_NAME) = 'onion'

In [0]:
-- update silver.daily_pricing_silver
-- set productgroup_name = 'hybrid vegitables',
--     record_updated_date = current_timestamp()
-- where lower(product_name) = 'onion'

In [0]:
show create table uc_prod.gold.reporting_dim_product_gold

In [0]:
-- CREATE TABLE uc_prod.gold.reporting_dim_product_gold_scd2 (
--   PRODUCTGROUP_NAME STRING,
--   PRODUCT_NAME STRING,
--   PRODUCT_ID BIGINT,
--   start_date TIMESTAMP,
--   end_date TIMESTAMP,
--   lakehouse_inserted_date TIMESTAMP,
--   lakehouse_updated_date TIMESTAMP)
-- USING delta

### Getting the distinct states from the staing master table (silver.daily_pricing_silver)

In [0]:

create or replace table silver.dim_state_staging_st1_sc1
select distinct productgroup_name,product_name from silver.daily_pricing_silver
where record_updated_date > (
      select nvl(max(process_file_updated_date),'2010-01-01') from uc_prod.watermark_tbl.process_logs 
      where process_name = 'reporting_dim_product_gold_scd2' and process_status ='completed'
                            )


In [0]:
select * from silver.dim_state_staging_st1_sc1

### Add the distinct unique id for the state items

In [0]:
create or replace table silver.dim_state_staging_st2
select 
st1.product_name,st1.productgroup_name,
gpr.PRODUCT_ID as gold_product_id,
row_number()over(order by st1.productgroup_name,st1.product_name)  rn_num,
case when gpr.PRODUCT_ID is null then 'new' else 'changed' end as max_flag
from silver.dim_state_staging_st1_sc1 as st1
left join uc_prod.gold.reporting_dim_product_gold_scd2 as gpr
on lower(gpr.PRODUCT_NAME) = lower(st1.product_name) and gpr.end_date is null
where gpr.PRODUCT_NAME is null or st1.productgroup_name <> gpr.PRODUCTGROUP_NAME


In [0]:
select * from silver.dim_state_staging_st2

In [0]:
create or replace table silver.dim_state_staging_st3
select  (t.rn_num+s.max_flag) as  product_id,
t.product_name as product_name,
t.max_flag,
t.gold_product_id,
t.productgroup_name as productgroup_name
,current_timestamp() as lakehouse_inserted_date,
current_timestamp() as lakehouse_updated_date
from silver.dim_state_staging_st2 as t
cross join (select nvl(max(PRODUCT_ID),0) as max_flag from uc_prod.gold.reporting_dim_product_gold_scd2 ) as s

In [0]:
select * from silver.dim_state_staging_st3 where lower(product_name) = 'onion' 

In [0]:
select * from uc_prod.gold.reporting_dim_product_gold_scd2

In [0]:
merge into uc_prod.gold.reporting_dim_product_gold_scd2 as tgt
using (
select * from silver.dim_state_staging_st3 as t
) as src
on tgt.PRODUCT_ID = src.gold_product_id
when matched then update 
  set tgt.lakehouse_updated_date = current_timestamp(),
      tgt.end_date = current_timestamp()
when not matched then  
  insert (
      PRODUCT_ID,
      PRODUCT_NAME,
      PRODUCTGROUP_NAME,
      start_date,
      end_date,
      lakehouse_inserted_date,
      lakehouse_updated_date
         )
  values
        (
        src.product_id,
        src.product_name,
        src.productgroup_name,
        current_timestamp(),
        null,
        src.lakehouse_inserted_date,
        src.lakehouse_updated_date
        )


In [0]:
select * from uc_prod.gold.reporting_dim_product_gold_scd2

In [0]:
insert into uc_prod.gold.reporting_dim_product_gold_scd2
select 
productgroup_name,
product_name,
product_id,
current_timestamp(),
null,
current_timestamp(),
current_timestamp()
from silver.dim_state_staging_st3
where max_flag = 'changed'

### update the water mark table

In [0]:
select * from gold.reporting_dim_product_gold_scd2 where end_date is not null

In [0]:
insert into uc_prod.watermark_tbl.process_logs
(Process_name,Process_file_updated_date,process_status)
values
(
'reporting_dim_product_gold_scd2',
current_timestamp(),
'completed'
)

In [0]:
--truncate table uc_prod.gold.reporting_dim_product_gold_scd2;
-- delete from uc_prod.watermark_tbl.process_logs where process_name = 'reporting_dim_product_gold_scd2'


In [0]:
select * from gold.reporting_dim_product_gold_scd2 where lower(product_name) = 'onion'